# Chapter 18: Transactions and Database Patterns

Beyond basic CRUD operations, real-world database code needs reliable transaction
management, convenient row access patterns, and type handling for Python objects.
This notebook covers the patterns that make `sqlite3` code production-ready.

## What You Will Learn
- Transaction control: `BEGIN`, `COMMIT`, `ROLLBACK`
- Using `Connection` as a context manager for automatic commit/rollback
- Isolation levels: `DEFERRED`, `IMMEDIATE`, `EXCLUSIVE`
- `sqlite3.Row` for dict-like access to query results
- Custom row factories
- Type adapters and converters (dates, custom types)
- Practical pattern: configuration store with upsert

## Transactions: BEGIN, COMMIT, ROLLBACK

A **transaction** groups multiple SQL operations into an atomic unit: either
all succeed (`COMMIT`) or all are undone (`ROLLBACK`). SQLite starts transactions
implicitly, but you can control them explicitly for safety.

In [ ]:
import sqlite3

conn: sqlite3.Connection = sqlite3.connect(":memory:")
cur: sqlite3.Cursor = conn.cursor()

cur.execute("""
    CREATE TABLE accounts (
        id      INTEGER PRIMARY KEY,
        name    TEXT NOT NULL,
        balance REAL NOT NULL DEFAULT 0.0
    )
""")
cur.executemany(
    "INSERT INTO accounts (name, balance) VALUES (?, ?)",
    [("Alice", 1000.0), ("Bob", 500.0)],
)
conn.commit()


def show_balances(connection: sqlite3.Connection) -> None:
    """Print all account balances."""
    for name, balance in connection.execute("SELECT name, balance FROM accounts"):
        print(f"  {name}: ${balance:.2f}")


print("Initial balances:")
show_balances(conn)

In [ ]:
# Simulate a bank transfer with explicit transaction control
transfer_amount: float = 200.0

try:
    cur.execute("UPDATE accounts SET balance = balance - ? WHERE name = ?",
                (transfer_amount, "Alice"))
    cur.execute("UPDATE accounts SET balance = balance + ? WHERE name = ?",
                (transfer_amount, "Bob"))
    conn.commit()  # Both updates succeed together
    print("Transfer committed successfully!")
except Exception as e:
    conn.rollback()  # Undo both updates on any error
    print(f"Transfer rolled back: {e}")

print("\nAfter transfer:")
show_balances(conn)

In [ ]:
# Demonstrate ROLLBACK: simulate a failed transfer
try:
    cur.execute("UPDATE accounts SET balance = balance - ? WHERE name = ?",
                (5000.0, "Alice"))  # Would overdraw

    # Check constraint manually (SQLite lacks CHECK for this demo)
    cur.execute("SELECT balance FROM accounts WHERE name = ?", ("Alice",))
    alice_balance: float = cur.fetchone()[0]
    if alice_balance < 0:
        raise ValueError(f"Insufficient funds: Alice has ${alice_balance:.2f}")

    conn.commit()
except (ValueError, sqlite3.Error) as e:
    conn.rollback()
    print(f"Transaction rolled back: {e}")

print("\nBalances unchanged after rollback:")
show_balances(conn)

## Connection as Context Manager

The recommended pattern is to use `conn` as a context manager in a `with` block.
On success, it auto-commits; on exception, it auto-rolls back. This eliminates
the try/except boilerplate.

**Important**: The context manager handles `COMMIT`/`ROLLBACK`, but it does
**not** close the connection. You still need to close it yourself.

In [ ]:
# Successful transaction -- auto-committed
with conn:
    conn.execute("UPDATE accounts SET balance = balance - ? WHERE name = ?",
                 (100.0, "Alice"))
    conn.execute("UPDATE accounts SET balance = balance + ? WHERE name = ?",
                 (100.0, "Bob"))

print("Auto-committed transfer:")
show_balances(conn)

# Failed transaction -- auto-rolled back
try:
    with conn:
        conn.execute("UPDATE accounts SET balance = balance + ? WHERE name = ?",
                     (1000000.0, "Bob"))
        raise RuntimeError("Something went wrong mid-transaction!")
except RuntimeError as e:
    print(f"\nCaught error: {e}")

print("Balances after auto-rollback (Bob's million reverted):")
show_balances(conn)

## Isolation Levels

SQLite supports three transaction isolation levels that control **when** a
transaction acquires database locks:

| Level | Lock Acquired | Use Case |
|-------|--------------|----------|
| `DEFERRED` (default) | On first read/write | General use |
| `IMMEDIATE` | On `BEGIN` | Writes that need early lock |
| `EXCLUSIVE` | On `BEGIN`, blocks all | Full exclusive access |

In Python's `sqlite3`, set the isolation level via the `isolation_level` parameter
to `connect()`, or set it to `None` for **autocommit mode** (each statement is
its own transaction).

In [ ]:
# Default isolation level
print(f"Current isolation_level: {conn.isolation_level!r}")

# Create a new connection with explicit isolation level
conn_immediate: sqlite3.Connection = sqlite3.connect(
    ":memory:", isolation_level="IMMEDIATE"
)
print(f"IMMEDIATE connection: {conn_immediate.isolation_level!r}")
conn_immediate.close()

# Autocommit mode: isolation_level=None
conn_auto: sqlite3.Connection = sqlite3.connect(
    ":memory:", isolation_level=None
)
print(f"Autocommit connection: {conn_auto.isolation_level!r}")

# In autocommit mode, each statement commits immediately
conn_auto.execute("CREATE TABLE demo (val TEXT)")
conn_auto.execute("INSERT INTO demo VALUES (?)", ("auto-committed",))
result = conn_auto.execute("SELECT val FROM demo").fetchone()
print(f"Auto-committed value: {result[0]}")
conn_auto.close()

## sqlite3.Row: Dict-Like Row Access

By default, `sqlite3` returns rows as plain tuples. Setting
`conn.row_factory = sqlite3.Row` gives you objects that support both
index-based **and** name-based access, plus `keys()` for column names.

In [ ]:
# Enable sqlite3.Row for dict-like access
conn.row_factory = sqlite3.Row
cur = conn.cursor()

cur.execute("SELECT id, name, balance FROM accounts")
rows: list[sqlite3.Row] = cur.fetchall()

for row in rows:
    # Access by column name
    print(f"Name: {row['name']}, Balance: ${row['balance']:.2f}")

    # Access by index still works
    assert row[1] == row['name']

# Get column names
print(f"\nColumn names: {rows[0].keys()}")

# Convert to a regular dict
account_dict: dict[str, object] = dict(rows[0])
print(f"As dict: {account_dict}")

## Custom Row Factories

A **row factory** is a callable that receives `(cursor, row_tuple)` and returns
whatever representation you prefer: dicts, dataclasses, named tuples, etc.

In [ ]:
from dataclasses import dataclass


@dataclass
class Account:
    """Represents a bank account."""
    id: int
    name: str
    balance: float


def account_factory(cursor: sqlite3.Cursor, row: tuple) -> Account:
    """Row factory that returns Account dataclass instances."""
    return Account(*row)


# Apply the custom row factory
conn.row_factory = account_factory
cur = conn.cursor()

cur.execute("SELECT id, name, balance FROM accounts")
accounts: list[Account] = cur.fetchall()

for acct in accounts:
    print(f"{acct.name}'s balance: ${acct.balance:.2f} (type: {type(acct).__name__})")

# A generic dict factory
def dict_factory(cursor: sqlite3.Cursor, row: tuple) -> dict[str, object]:
    """Row factory that returns plain dictionaries."""
    columns: list[str] = [col[0] for col in cursor.description]
    return dict(zip(columns, row))

conn.row_factory = dict_factory
result = conn.execute("SELECT id, name, balance FROM accounts").fetchall()
print(f"\nAs dicts: {result}")

# Reset to default
conn.row_factory = None

## Type Adapters and Converters

SQLite has limited native types (`INTEGER`, `REAL`, `TEXT`, `BLOB`, `NULL`).
Python's `sqlite3` module uses **adapters** and **converters** to bridge
between Python types and SQLite storage:

- **Adapter**: Python object -> SQLite-compatible value (on write)
- **Converter**: SQLite value -> Python object (on read)

`datetime` support is built in when you use `detect_types`.

In [ ]:
import sqlite3
from datetime import date, datetime

# Enable built-in date/datetime converters
conn_typed: sqlite3.Connection = sqlite3.connect(
    ":memory:",
    detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES,
)

conn_typed.execute("""
    CREATE TABLE events (
        id         INTEGER PRIMARY KEY,
        name       TEXT NOT NULL,
        event_date date,
        created_at timestamp
    )
""")

# Insert Python date and datetime objects -- adapted automatically
conn_typed.execute(
    "INSERT INTO events (name, event_date, created_at) VALUES (?, ?, ?)",
    ("Conference", date(2025, 6, 15), datetime(2025, 1, 10, 9, 30, 0)),
)
conn_typed.commit()

# Read them back -- converted automatically to Python types
row = conn_typed.execute("SELECT name, event_date, created_at FROM events").fetchone()
name, event_date, created_at = row

print(f"Event:      {name}")
print(f"Date:       {event_date} (type: {type(event_date).__name__})")
print(f"Created at: {created_at} (type: {type(created_at).__name__})")

In [ ]:
import json


# Custom adapter and converter for storing JSON in SQLite
class Settings:
    """A custom type that stores structured settings."""

    def __init__(self, data: dict[str, object]) -> None:
        self.data = data

    def __repr__(self) -> str:
        return f"Settings({self.data})"


# Adapter: Settings -> TEXT (for storing in SQLite)
def adapt_settings(settings: Settings) -> str:
    return json.dumps(settings.data)


# Converter: TEXT -> Settings (for reading from SQLite)
def convert_settings(raw: bytes) -> Settings:
    return Settings(json.loads(raw.decode()))


# Register the adapter and converter
sqlite3.register_adapter(Settings, adapt_settings)
sqlite3.register_converter("settings", convert_settings)

# Use the custom type
conn_custom: sqlite3.Connection = sqlite3.connect(
    ":memory:", detect_types=sqlite3.PARSE_DECLTYPES
)
conn_custom.execute("CREATE TABLE prefs (id INTEGER PRIMARY KEY, config settings)")
conn_custom.execute(
    "INSERT INTO prefs (config) VALUES (?)",
    (Settings({"theme": "dark", "font_size": 14, "lang": "en"}),),
)
conn_custom.commit()

row = conn_custom.execute("SELECT config FROM prefs").fetchone()
settings: Settings = row[0]
print(f"Retrieved: {settings}")
print(f"Type:      {type(settings).__name__}")
print(f"Theme:     {settings.data['theme']}")

conn_custom.close()
conn_typed.close()

## Practical Pattern: Configuration Store with Upsert

An **upsert** (INSERT or UPDATE) is a common pattern for key-value stores.
SQLite 3.24+ supports `ON CONFLICT` for this. Here we build a reusable
configuration store.

In [ ]:
import sqlite3


class ConfigStore:
    """A persistent key-value configuration store backed by SQLite."""

    def __init__(self, db_path: str = ":memory:") -> None:
        self.conn: sqlite3.Connection = sqlite3.connect(db_path)
        self.conn.execute("""
            CREATE TABLE IF NOT EXISTS config (
                key   TEXT PRIMARY KEY,
                value TEXT NOT NULL
            )
        """)
        self.conn.commit()

    def set(self, key: str, value: str) -> None:
        """Set a configuration value (insert or update)."""
        with self.conn:
            self.conn.execute(
                "INSERT INTO config (key, value) VALUES (?, ?) "
                "ON CONFLICT(key) DO UPDATE SET value = excluded.value",
                (key, value),
            )

    def get(self, key: str, default: str | None = None) -> str | None:
        """Get a configuration value, or return default if not found."""
        row = self.conn.execute(
            "SELECT value FROM config WHERE key = ?", (key,)
        ).fetchone()
        return row[0] if row else default

    def delete(self, key: str) -> bool:
        """Delete a configuration key. Returns True if the key existed."""
        with self.conn:
            cur = self.conn.execute("DELETE FROM config WHERE key = ?", (key,))
        return cur.rowcount > 0

    def all(self) -> dict[str, str]:
        """Return all configuration as a dictionary."""
        rows = self.conn.execute("SELECT key, value FROM config ORDER BY key").fetchall()
        return dict(rows)

    def close(self) -> None:
        self.conn.close()


# Use the configuration store
store = ConfigStore()

store.set("app.name", "MyApp")
store.set("app.version", "1.0.0")
store.set("app.debug", "true")
print(f"All config: {store.all()}")

# Upsert: update an existing key
store.set("app.version", "2.0.0")
print(f"\nAfter upsert version: {store.get('app.version')}")

# Get with default
print(f"Missing key: {store.get('app.secret', 'not-set')}")

# Delete
deleted: bool = store.delete("app.debug")
print(f"\nDeleted 'app.debug': {deleted}")
print(f"Final config: {store.all()}")

store.close()

## Summary

### Key Takeaways

| Topic | What You Learned |
|-------|------------------|
| **Transactions** | Group operations atomically with `COMMIT`/`ROLLBACK` |
| **Context manager** | `with conn:` auto-commits on success, rolls back on exception |
| **Isolation levels** | `DEFERRED`, `IMMEDIATE`, `EXCLUSIVE`, or `None` for autocommit |
| **sqlite3.Row** | Dict-like row access via `conn.row_factory = sqlite3.Row` |
| **Custom factories** | Return dataclasses, dicts, or any type from queries |
| **Adapters/converters** | Bridge Python types to/from SQLite storage (dates, JSON, etc.) |
| **Upsert pattern** | `INSERT ... ON CONFLICT DO UPDATE` for key-value stores |

### Best Practices
- **Use `with conn:`** for all write operations to ensure atomic transactions
- **Set `row_factory`** early to avoid manual tuple unpacking throughout your code
- **Register adapters/converters** for domain types to keep SQL code clean
- **Use `detect_types`** with `PARSE_DECLTYPES` for automatic type handling
- **Prefer upsert** (`ON CONFLICT`) over check-then-insert patterns to avoid race conditions